# 04 — Performance Profiling

Profile SQL queries (before/after indexing) and Python code to demonstrate optimisation.

In [1]:
import sys, os, time, sqlite3, cProfile, pstats, io
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / "src").exists():
    sys.path.insert(0, str(PROJECT_ROOT))
else:
    PROJECT_ROOT = PROJECT_ROOT.parent
    sys.path.insert(0, str(PROJECT_ROOT))

from src.utils import *

## 4.1 Query Profiling — Setup

In [2]:
conn = get_db_connection()

queries = {
    "Q1: Count reviews per year": """
        SELECT year, COUNT(*) AS cnt
        FROM reviews GROUP BY year ORDER BY year
    """,
    "Q2: Top 10 hotels by avg rating (min 50 reviews)": """
        SELECT hotel_id, AVG(rating_overall) AS avg_r, COUNT(*) AS cnt
        FROM reviews GROUP BY hotel_id HAVING cnt >= 50
        ORDER BY avg_r DESC LIMIT 10
    """,
    "Q3: Monthly trends": """
        SELECT year, month, AVG(rating_overall) AS avg_r, COUNT(*) AS cnt
        FROM reviews GROUP BY year, month ORDER BY year, month
    """,
    "Q4: Filter by hotel and date range": """
        SELECT * FROM reviews
        WHERE hotel_id = (SELECT hotel_id FROM hotels ORDER BY num_reviews DESC LIMIT 1)
          AND date_parsed BETWEEN '2011-01-01' AND '2012-12-31'
    """,
    "Q5: Join hotels and reviews": """
        SELECT h.hotel_id, h.num_reviews,
               AVG(r.rating_overall) AS avg_r
        FROM hotels h JOIN reviews r ON h.hotel_id = r.hotel_id
        GROUP BY h.hotel_id
        ORDER BY avg_r DESC LIMIT 20
    """,
}

## 4.2 Query Profiling — With Indexes

In [3]:
results_with = {}
for name, sql in queries.items():
    # EXPLAIN QUERY PLAN
    plan = conn.execute(f"EXPLAIN QUERY PLAN {sql}").fetchall()
    plan_str = "\n".join(str(row) for row in plan)

    # Timing (average of 3 runs)
    times = []
    for _ in range(3):
        t0 = time.perf_counter()
        conn.execute(sql).fetchall()
        times.append(time.perf_counter() - t0)
    avg_t = sum(times) / len(times)

    results_with[name] = {"time": avg_t, "plan": plan_str}
    print(f"{name}")
    print(f"  Time: {avg_t*1000:.2f} ms")
    print(f"  Plan: {plan_str[:200]}")
    print()

Q1: Count reviews per year
  Time: 48.27 ms
  Plan: <sqlite3.Row object at 0x113d13fa0>

Q2: Top 10 hotels by avg rating (min 50 reviews)
  Time: 48735.11 ms
  Plan: <sqlite3.Row object at 0x113cdece0>

Q3: Monthly trends
  Time: 7797.14 ms
  Plan: <sqlite3.Row object at 0x1130ddde0>

Q4: Filter by hotel and date range
  Time: 56.51 ms
  Plan: <sqlite3.Row object at 0x113cdece0>

Q5: Join hotels and reviews
  Time: 27679.67 ms
  Plan: <sqlite3.Row object at 0x113d13280>



## 4.3 Query Profiling — Without Indexes

In [4]:
# Drop indexes temporarily
idx_names = [row[0] for row in conn.execute(
    "SELECT name FROM sqlite_master WHERE type='index' AND name LIKE 'idx_%'"
).fetchall()]
for idx in idx_names:
    conn.execute(f"DROP INDEX IF EXISTS {idx}")
conn.commit()
print(f"Dropped {len(idx_names)} indexes")

results_without = {}
for name, sql in queries.items():
    plan = conn.execute(f"EXPLAIN QUERY PLAN {sql}").fetchall()
    plan_str = "\n".join(str(row) for row in plan)
    times = []
    for _ in range(3):
        t0 = time.perf_counter()
        conn.execute(sql).fetchall()
        times.append(time.perf_counter() - t0)
    avg_t = sum(times) / len(times)
    results_without[name] = {"time": avg_t, "plan": plan_str}
    print(f"{name}")
    print(f"  Time (no index): {avg_t*1000:.2f} ms")
    print()

Dropped 6 indexes
Q1: Count reviews per year
  Time (no index): 9086.29 ms

Q2: Top 10 hotels by avg rating (min 50 reviews)
  Time (no index): 9185.25 ms

Q3: Monthly trends
  Time (no index): 9262.14 ms

Q4: Filter by hotel and date range
  Time (no index): 9721.60 ms

Q5: Join hotels and reviews
  Time (no index): 14341.10 ms



## 4.4 Restore Indexes & Comparison

In [5]:
# Restore indexes
schema_sql = (DATA_DIR / "data_schema.sql").read_text()
for line in schema_sql.splitlines():
    if line.strip().upper().startswith("CREATE INDEX"):
        conn.execute(line)
conn.commit()
print("Indexes restored.")

# Comparison table
rows = []
for name in queries:
    t_with = results_with[name]["time"] * 1000
    t_without = results_without[name]["time"] * 1000
    speedup = t_without / t_with if t_with > 0 else 0
    rows.append({"Query": name, "With Index (ms)": round(t_with, 2),
                 "No Index (ms)": round(t_without, 2),
                 "Speedup": f"{speedup:.1f}x"})
comparison = pd.DataFrame(rows)
print(comparison.to_string(index=False))

Indexes restored.
                                           Query  With Index (ms)  No Index (ms) Speedup
                      Q1: Count reviews per year            48.27        9086.29  188.2x
Q2: Top 10 hotels by avg rating (min 50 reviews)         48735.11        9185.25    0.2x
                              Q3: Monthly trends          7797.14        9262.14    1.2x
              Q4: Filter by hotel and date range            56.51        9721.60  172.0x
                     Q5: Join hotels and reviews         27679.67       14341.10    0.5x


## 4.5 Save Query Profiling Results

In [6]:
with open(PROFILING_DIR / "query_results.txt", "w") as f:
    f.write("Query Profiling Results")
    f.write("=" * 60 + "")
    for name in queries:
        f.write(f"{name}")
        f.write(f"  SQL: {queries[name].strip()[:200]}")
        f.write(f"  With indexes:    {results_with[name]["time"]*1000:.2f} ms")
        f.write(f"  Without indexes: {results_without[name]["time"]*1000:.2f} ms")
        speedup = results_without[name]["time"] / results_with[name]["time"] if results_with[name]["time"] > 0 else 0
        f.write(f"  Speedup: {speedup:.1f}x")
        f.write(f"  Query Plan (indexed):{results_with[name]["plan"]}")
    f.write(comparison.to_string(index=False))
print(f"Saved to {PROFILING_DIR / "query_results.txt"}")

Saved to /Users/adrian/Workshop/5126/Assignment1/profiling/query_results.txt


## 4.6 Code Profiling

In [7]:
from src.benchmarking import compute_hotel_features, cluster_hotels

pr = cProfile.Profile()
pr.enable()

# Profile the benchmarking pipeline
features = compute_hotel_features(min_reviews=10)
df_cl, sil, _, _ = cluster_hotels(features, n_clusters=5)

pr.disable()

# Capture profiling output
s = io.StringIO()
ps = pstats.Stats(pr, stream=s)
ps.sort_stats("cumulative")
ps.print_stats(30)
profile_output = s.getvalue()
print(profile_output[:3000])

         74151 function calls (73716 primitive calls) in 48.452 seconds

   Ordered by: cumulative time
   List reduced from 1013 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/3    0.000    0.000   40.430   13.477 /Users/adrian/Workshop/5126/Assignment1/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3665(run_code)
      4/3    0.000    0.000   40.430   13.477 {built-in method builtins.exec}
        1    0.000    0.000   39.998   39.998 /var/folders/7v/4q7r0xwd6tb59mlws6_tzcm40000gn/T/ipykernel_95713/1691772082.py:1(<module>)
        1    0.000    0.000   39.998   39.998 /Users/adrian/Workshop/5126/Assignment1/src/benchmarking.py:27(compute_hotel_features)
        1    9.059    9.059   39.936   39.936 /Users/adrian/Workshop/5126/Assignment1/.venv/lib/python3.13/site-packages/pandas/io/sql.py:436(read_sql_query)
       39   37.025    0.949   37.025    0.949 {built-in method time.sleep}
        7    0.00

## 4.7 Save Code Profiling Results

In [8]:
with open(PROFILING_DIR / "code_profiling.txt", "w") as f:
    f.write("Code Profiling Results")
    f.write("=" * 60 + "")
    f.write("Profiled: compute_hotel_features() + cluster_hotels()")
    f.write(profile_output)
print(f"Saved to {PROFILING_DIR / "code_profiling.txt"}")

conn.close()
print("Profiling complete ✓")

Saved to /Users/adrian/Workshop/5126/Assignment1/profiling/code_profiling.txt
Profiling complete ✓
